# 中文自然语言处理分析

by @福利

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个额外的分词工具去把完整的文本中分解成粒度更细的词。

In [40]:
import  pandas as pd
import  jieba
from jieba import analyse
import numpy as np
import sys

In [41]:
print(np.version)

<module 'numpy.version' from '/anaconda3/lib/python3.7/site-packages/numpy/version.py'>


In [42]:
sys.path

['',
 '/Users/zhengwenjie/AI/work/ML_3/NLP_project',
 '/anaconda3/lib/python37.zip',
 '/anaconda3/lib/python3.7',
 '/anaconda3/lib/python3.7/lib-dynload',
 '/anaconda3/lib/python3.7/site-packages',
 '/anaconda3/lib/python3.7/site-packages/aeosa',
 '/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/zhengwenjie/.ipython']

## 关键词提取

In [31]:
# 设置停止词
jieba.analyse.set_stop_words("./data/stopwords.txt")

* 基于TF－IDF 关键词提取

In [32]:
# 科技类新闻
news_data_path = "./data/technology_news.csv"
news_data_df = pd.read_csv(news_data_path,sep=",",encoding="utf-8",names=["id","content"],header=0)
news_data_df = news_data_df.dropna()
news_data_df["content"] = news_data_df.content.apply(lambda x:x.strip())
news_data_df.head()

,id,content
0,0,",中新网,1月7日电 恰逢CES 2017拉开大幕，却惊闻“AlphaGo升级版”的Mas..."
1,1,徐立，商汤科技CEO在谈起本次参展时谈到：“作为一个成立刚刚两年的创业公司，这次参展，一方面...
2,2,正如最后挑战Master的古力在落败后发表的观点：“人类与人工智能共同探索围棋世界的大幕即将...
3,3,", SenseFace人脸布控的“黑科技”"
4,4,本届大展最大的看点，无疑是“被誉为2016全美科技第一神股”英伟达的首次CES主题演讲。现场...


In [33]:
news_lst =  news_data_df.content.values.tolist()
lines = "".join(news_lst)

In [34]:
tfidf_tags = jieba.analyse.extract_tags(lines, topK=20, withWeight=True, allowPOS=())

In [35]:
print(tfidf_tags)

[('互联网', 0.03747879067428829), ('手机', 0.03478861572219171), ('平台', 0.034364016389595034), ('人工智能', 0.030207610612604942), ('百度', 0.029219914457491076), ('2017', 0.029031346517937033), ('智能', 0.028573966825895386), ('技术', 0.028374331540614292), ('数据', 0.027682957796662935), ('360', 0.026904049574812336), ('服务', 0.02684778784101862), ('直播', 0.02240843022914509), ('产品', 0.022358601482393867), ('企业', 0.021891067488386897), ('视频', 0.02054662325524849), ('网络', 0.018312896740235306), ('行业', 0.018203063456355206), ('游戏', 0.0180598440272288), ('机器人', 0.017653117461615065), ('电商', 0.01757777596306257)]


In [36]:
# 军事类新闻
news_data_path = "./data/military_news.csv"
news_data_df = pd.read_csv(news_data_path,sep=",",encoding="utf-8",names=["id","content"],header=0)
news_data_df = news_data_df.dropna()
news_data_df["content"] = news_data_df.content.apply(lambda x:x.strip())
print(news_data_df.head())
news_lst =  news_data_df.content.values.tolist()
lines = "".join(news_lst)
tfidf_tags = jieba.analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=['ns', 'n', 'vn', 'v','nr'])
print(tfidf_tags)

   id                                            content
0   0  新华社北京1月8日电(记者董瑞丰)军民融合发展已经上升为国家战略，但现阶段国防工业企业军民融...
1   1  中国国防工业企业军民融合产业联盟8日在北京宣布成立。刘东奎说，适时成立联盟，坚持“军民融合、...
2   2  数据显示，我国目前有十几万家民营高新技术企业，很多高新技术企业在新能源、新材料、电子信息产业...
3   3  对于现阶段军民融合面临的问题，刘东奎列举：从企业外部来看，军民融合管理体制不够顺畅，军民融合...
4   4  “国防科技工业是军民融合的天然载体和核心领域。”刘东奎说，未来要着力搭建军民协同创新展示引导...
['航母', '训练', '海军', '中国', '官兵', '部队', '编队', '作战', '美国', '导弹', '能力', '军事', '装备', '军队', '保障', '战略', '军人', '日本', '南海', '战机']


* 基于textrank 关键词企图

In [43]:
textrank_tags = jieba.analyse.textrank(lines, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))
print(textrank_tags)

['中国', '海军', '训练', '美国', '部队', '官兵', '航母', '作战', '能力', '军事', '发展', '工作', '国家', '建设', '导弹', '编队', '日本', '技术', '装备', '保障']


## LDA 主体模型

咱们来用LDA主题模型建模，看看这些新闻主要在说哪些topic。

首先我们要把文本内容处理成固定的格式，一个包含句子的list，list中每个元素是分词后的词list。类似下面这个样子。

[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]

In [134]:
from gensim import corpora,models,similarities
import gensim

In [177]:
news_data_path = "./data/technology_news.csv"
news_data_df = pd.read_csv(news_data_path, sep=",", encoding="utf-8", names=["id", "content"], header=0)
news_data_df = news_data_df.dropna()
news_data_df["content"] = news_data_df.content.apply(lambda x: x.strip())
print(news_data_df.head())
## 去除停用词
stopwords_data_path = "data/stopwords.txt"
stopwords_data_df = pd.read_csv(stopwords_data_path,encoding="utf-8",sep="\t",index_col=None,quoting=3,names=["stopword"])
print(stopwords_data_df.head())

stopwords_list = stopwords_data_df.stopword.tolist()
print(stopwords_list[0:4])


   id                                            content
0   0  ,中新网,1月7日电  恰逢CES 2017拉开大幕，却惊闻“AlphaGo升级版”的Mas...
1   1  徐立，商汤科技CEO在谈起本次参展时谈到：“作为一个成立刚刚两年的创业公司，这次参展，一方面...
2   2  正如最后挑战Master的古力在落败后发表的观点：“人类与人工智能共同探索围棋世界的大幕即将...
3   3                              ,　SenseFace人脸布控的“黑科技”
4   4  本届大展最大的看点，无疑是“被誉为2016全美科技第一神股”英伟达的首次CES主题演讲。现场...
  stopword
0        !
1        "
2        #
3        $
4        %
['!', '"', '#', '$']


In [178]:
lines = news_data_df.content.tolist()

In [179]:
sentences=[]
for line in lines:
    try:
        segs = jieba.lcut(line)        
        segs_list=[]
        for seg in segs:
            if(len(seg) > 1) and (seg not in stopwords_list):
                segs_list.append(seg)
        sentences.append(segs_list)
    except Exception:
        continue

In [180]:
sentences[0:2]

[['中新网',
  '日电',
  'CES',
  '2017',
  '拉开',
  '大幕',
  '惊闻',
  'AlphaGo',
  '升级版',
  'Master',
  '迎来',
  '连胜',
  '人类',
  '顶尖',
  '围棋',
  '一周',
  '纷纷',
  '败给',
  '谷歌',
  '旗下',
  'DeepMind',
  '团队',
  '打造',
  '围棋',
  '大脑',
  '聚焦',
  '人工智能',
  '本届',
  'CES',
  '增添',
  '声势',
  '首次',
  '参展',
  '致力于',
  '打造',
  '原创',
  'AI',
  '大脑',
  '中国',
  '深度',
  '学习',
  '领军',
  '企业',
  '商汤',
  '科技',
  '人工智能',
  '浪潮',
  '之巅',
  '众多',
  '业界',
  '前辈',
  '巨匠',
  '面前',
  '将会',
  '交出',
  '一份',
  '答卷'],
 ['徐立',
  '商汤',
  '科技',
  'CEO',
  '谈起',
  '本次',
  '参展',
  '谈到',
  '成立',
  '刚刚',
  '两年',
  '创业',
  '公司',
  '参展',
  '展示',
  '最新',
  '人工智能',
  '技术',
  '产品',
  '表达',
  '人工智能',
  '理解',
  '人工智能',
  '特定',
  '领域',
  '超越',
  '人类',
  '广泛应用',
  '标志',
  'Master',
  '胜利',
  '围棋',
  '世界',
  '开拓',
  '局面',
  '不谋而合']]

In [181]:
# 词典模型
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [182]:
# 基于词袋模型训练LDA 模型
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=20)

In [193]:
# lda 模型保持
from gensim.test.utils import datapath
temp_file = datapath("lda_20190222.model")
lda.save(temp_file)

In [195]:
from gensim.models import LdaModel
# lda 模型加载
lda_new = LdaModel.load(temp_file)
# 查看第三号分类，以及常出现的词语
lda_new.print_topic(3,topn=10)

'0.068*"游戏" + 0.011*"地图" + 0.011*"玩家" + 0.011*"设计" + 0.009*"电子竞技" + 0.008*"充电" + 0.007*"青年" + 0.007*"独立" + 0.007*"电竞" + 0.007*"采用"'

In [204]:
for topic in lda_new.print_topics(num_topics=20,num_words=10):
    print(str(topic[0]) + "---->" + topic[1])

0---->0.023*"网络" + 0.013*"诈骗" + 0.013*"企业" + 0.008*"电信" + 0.008*"保护" + 0.007*"支付" + 0.006*"平台" + 0.006*"体系" + 0.006*"风险" + 0.006*"相关"
1---->0.016*"创业" + 0.009*"项目" + 0.008*"中国" + 0.007*"举办" + 0.006*"媒体" + 0.006*"发布会" + 0.006*"平台" + 0.006*"论坛" + 0.006*"活动" + 0.006*"2017"
2---->0.029*"技术" + 0.023*"科技" + 0.019*"发展" + 0.014*"公司" + 0.014*"中国" + 0.014*"全球" + 0.014*"领域" + 0.013*"行业" + 0.012*"创新" + 0.011*"未来"
3---->0.068*"游戏" + 0.011*"地图" + 0.011*"玩家" + 0.011*"设计" + 0.009*"电子竞技" + 0.008*"充电" + 0.007*"青年" + 0.007*"独立" + 0.007*"电竞" + 0.007*"采用"
4---->0.038*"视频" + 0.036*"内容" + 0.018*"平台" + 0.016*"腾讯" + 0.015*"勒索" + 0.012*"广告" + 0.012*"流量" + 0.009*"营销" + 0.009*"APP" + 0.007*"媒体"
5---->0.042*"手机" + 0.023*"市场" + 0.023*"产品" + 0.021*"品牌" + 0.015*"消费者" + 0.012*"小米" + 0.010*"电视" + 0.007*"厂商" + 0.007*"体验" + 0.006*"智能手机"
6---->0.014*"生活" + 0.013*"分享" + 0.013*"互联网" + 0.008*"空间" + 0.008*"金立" + 0.008*"需求" + 0.008*"方式" + 0.007*"城市" + 0.007*"平台" + 0.007*"工作"
7---->0.030*"中国" + 0.017*"数据" + 0.015*"发展" + 0.014*"

In [206]:
# 对新加入的文本，进行简单主题分类：
text = "国产厂商选择印度作为出海第一站的一个重要原因就是专利门槛相对较低。不过，近年来国产手机厂商在印度面临多起专利诉讼。杜比实验室(Dolby)在印度起诉了OPPO、vivo两家中国智能手机厂商，小米因涉嫌侵犯技术专利被爱立信起诉至印度德里高等法院，爱立信申请“临时禁令”并获同意，红米Note和红米1S手机暂停销售。"
corpus = dictionary.doc2bow(jieba.lcut(text))
print(corpus)

lda_new.get_document_topics(corpus)



[(9, 1), (66, 1), (88, 1), (93, 1), (96, 1), (180, 1), (209, 1), (232, 1), (267, 1), (275, 1), (325, 3), (329, 1), (330, 4), (331, 3), (332, 1), (345, 1), (347, 1), (348, 1), (349, 1), (679, 1), (680, 1), (681, 1), (682, 1), (683, 1), (684, 1), (685, 1), (686, 1), (687, 1), (688, 1), (689, 1), (690, 2), (691, 1), (692, 1), (693, 2), (694, 1), (695, 2), (696, 1), (697, 1)]


[(5, 0.31276804), (9, 0.60823065), (14, 0.039259266), (17, 0.02341554)]